<h1>Preliminar setup</h1>

In [1]:
import yfinance as yf
import ast 
import pandas as pd
from datetime import datetime
import os
import numpy as np
import matplotlib.pyplot as plt

european = ['^SPX', '^NDX', '^RUT']
#european = ['^NDX']

american = ['NVDA', 'JNJ', 'XOM']

#Parametric string
opt_filename = './data/options_daily/raw/{date_dir}/{date_file}_{title}_{type}.csv'

opt_filename_proc = './data/options_daily/proc/{date_dir}/{date_file}_{title}_{type}.csv'

title_filename = './data/title/{title}.csv'

#List of dates day by day from 2024_11_12 to 2024_11_29 
dates = pd.date_range(start='2024-11-11', end='2024-11-29').strftime('%Y_%m_%d').tolist()

<h1>Scrape title data</h1>

In [9]:
def scrape_title_data(title, start_date, end_date):
    stock = yf.Ticker(title)
    historical_data = stock.history(start=start_date, end=end_date)
    #Add column log ret given by ln(close_price(t))-ln(close_price(t-1))
    historical_data['log_ret'] = np.log(historical_data['Close']) - np.log(historical_data['Close'].shift(1))
    #remove first row
    historical_data = historical_data.iloc[1:]
    historical_data.to_csv(title_filename.format(title=title))

In [ ]:
start_date = "2021-12-01"
end_date = "2024-12-02"

for title in american + european:
    print(f"Scraping {title}")
    scrape_title_data(title, start_date, end_date)
print("Done")

<h1>Scrape Options Data</h1>

In [ ]:
def scrape_options_data(options, today):
    
    for idx in options:
        spx = yf.Ticker(idx)

        # get option chain for specific expiration
        try:
            opt = spx.option_chain('0000-00-00')
        except Exception as e:
            list_string = "[" + str(e).split('[')[1]
            list_string = list_string.replace(" ", "")
            list_string = list_string.replace(",", "','")
            list_string = list_string.replace("[", "['")
            list_string = list_string.replace("]", "']")
            option_dates = ast.literal_eval(list_string)
        
        all_calls = pd.DataFrame()
        all_puts = pd.DataFrame()
        
        # Define the cutoff date
        cutoff_date = datetime(2024, 12, 31)
        
        for date in option_dates:
            # Convert date to a datetime object if it's not already one
            if isinstance(date, str):
                date_obj = datetime.strptime(date, '%Y-%m-%d')
            
            if date_obj < cutoff_date:
                opt = spx.option_chain(date)
                
                #Process calls
                call = opt.calls
                call['expiration_date'] = date #add expiration date to the dataframe
                all_calls = pd.concat([all_calls, call], ignore_index=True)
                #all_calls = all_calls[all_calls.isna().sum(axis=1) <= 1]
                #all_calls = all_calls.dropna()
                
                #Process puts
                put = opt.puts
                put['expiration_date'] = date #add expiration_date to the dataframe
                all_puts = pd.concat([all_puts, put], ignore_index=True)
                #all_puts = all_puts[all_puts.isna().sum(axis=1) <= 1]
                #all_puts = all_puts.dropna()
        
        #If doesn't exist, create a data folder
        all_calls.to_csv('./data/options_daily/raw/' + today + '/' + today + '_' + idx + '_calls.csv', index=False)
        all_puts.to_csv('./data/options_daily/raw/' + today + '/' + today + '_' + idx + '_puts.csv', index=False)

In [ ]:
#Get today date in format yyyy_mm_dd
today = pd.Timestamp.today().strftime('%Y_%m_%d')

try:
    os.makedirs('./data/options_daily/raw/' + today)
except Exception as e:
    print('Data already written for today')
    exit()

print('Scraping European options data')
scrape_options_data(european, today)

print('Scraping American options data')
scrape_options_data(american, today)
    
print('Scraping completed')

<h1>Take only data until 29/11/2024 </h1>

In [ ]:
#For all datasets take only the rows with expiration_date until 2024-11-29
for date in dates:
    for idx in european + american:
        for option_type in ['calls', 'puts']:
            df = pd.read_csv(opt_filename.format(date_dir=date, date_file=date, title=idx, type=option_type))
            df['expiration_date'] = pd.to_datetime(df['expiration_date'])
            df = df[df['expiration_date'] <= '2024-11-29']
            df.to_csv(opt_filename_proc.format(date_dir=date, date_file=date, title=idx, type=option_type), index=False)

print('Done')


<h1>Take for every day only the expiration dates contained in all files (Intersection)</h1>

In [ ]:
from functools import reduce

dates_lists = []

#Build a list made of a list for every day sampled, for each day build a list, for each title, containing all the expiration dates for that title
for date in dates:
    day_dates_lists = []
    for idx in european + american:
        option_dates_list = []
        for option_type in ['calls', 'puts']:
            df = pd.read_csv(opt_filename_proc.format(date_dir=date, date_file=date, title=idx, type=option_type))
            option_dates_list.extend(df['expiration_date'].unique())
        day_dates_lists.append(set(option_dates_list)) #Set to remove duplicates from the add of the same dates in put and call dataset
            
    dates_lists.append(day_dates_lists)

#Note that there is no 2024-11-28 cause it's Thanksgiving
#print(len(dates_lists)) #Expected 19 as we have sampled 19 days
#print(len(dates_lists[0])) #Expected 6 as we have 6 titles
#print(len(dates_lists[0][0])) #Expected X

#Now take the intersection of the expiration dates for each title in each day
intersection_lists = []
for day_lists in dates_lists:
    intersection = list(reduce(lambda x, y: set(x) & set(y), day_lists))
    print(len(intersection))
    intersection_lists.append(intersection)
print(len(intersection_lists)) #Expected 19 as we have sampled 19 days
print(intersection_lists)
print('Done')

<h1>Take only data in intersection list of erxpirations</h1>

In [ ]:
#For all datasets take only the dates in the instersection list
for i in range(0, len(dates)):
    for idx in european + american:
        for option_type in ['calls', 'puts']:
            df = pd.read_csv(opt_filename_proc.format(date_dir=dates[i], date_file=dates[i], title=idx, type=option_type))
            df = df[df['expiration_date'].isin(intersection_lists[i])]
            df.to_csv(opt_filename_proc.format(date_dir=dates[i], date_file=dates[i], title=idx, type=option_type), index=False)

print('Done')

<h1>Take for each day only the put and calls with the same last trade</h1>

<h1>Calcolo del tasso privo di rischio</h1>

In [23]:
#Calcolo la media dei rendimenti

tb_dates = ['2024']

#create a dateset concatenating each year
df_all = pd.DataFrame()
for date in tb_dates:
    df = pd.read_csv(f'./data/bond/daily-treasury-rates_{date}.csv')
    df_all = pd.concat([df_all, df], ignore_index=True)

#Take only the elements in Date column which starts with 11/
df_all = df_all[df_all['Date'].str.startswith('11/')]

#drop date column
df_all = df_all.drop(columns=['Date'])

#Take mean skipping nan values
means = df_all.mean(skipna=True)

df_means = pd.DataFrame(means).T

#output to csv
df_means.to_csv('./data/bond/daily-treasury-rates_mean.csv', index=False)


'''
4 WEEKS BANK DISCOUNT,4 WEEKS COUPON EQUIVALENT,8 WEEKS BANK DISCOUNT,8 WEEKS COUPON EQUIVALENT,13 WEEKS BANK DISCOUNT,13 WEEKS COUPON EQUIVALENT,17 WEEKS BANK DISCOUNT,17 WEEKS COUPON EQUIVALENT,26 WEEKS BANK DISCOUNT,26 WEEKS COUPON EQUIVALENT,52 WEEKS BANK DISCOUNT,52 WEEKS COUPON EQUIVALENT
4.529473684210527,4.609473684210527,4.483157894736842,4.574210526315789,4.418421052631579,4.528421052631579,4.392631578947368,4.522631578947369,4.306842105263157,4.463684210526317,4.14421052631579,4.334736842105263
'''

<h1>Calcolo della volatilitá di lungo periodo</h1>

In [20]:
#Dato che i titoli nel mercato sono autocorrelati e eteroschedastici cioé hanno varianza variabile posso usare un modello garch per calcolare la volatilitá di lungo periodo
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
import pandas as pd

# Import the fGarch library in R
ro.r("""
if (!require(fGarch)) install.packages("fGarch", repos="http://cran.r-project.org")
library(fGarch)

# Carica anche i pacchetti richiesti per ridurre l'avviso
if (!require(fBasics)) install.packages("fBasics", repos="http://cran.r-project.org")
if (!require(timeDate)) install.packages("timeDate", repos="http://cran.r-project.org")
if (!require(timeSeries)) install.packages("timeSeries", repos="http://cran.r-project.org")
if (!require(Metrics)) install.packages("Metrics", repos="http://cran.r-project.org")

library(fBasics)
library(timeDate)
library(timeSeries)
library(Metrics)
""")

pandas2ri.activate()

def compute_long_run_volatility(title):
    df = pd.read_csv(title_filename.format(title=title))
    
    # Prendo come training set tutti i dati fino al 31 ottobre 2024
    r_data_training = df[df['Date'] <= '2024-10-31']['log_ret']
    r_data_testing= df[df['Date'] > '2024-10-31']['log_ret']


    # Converti la Serie Pandas in un DataFrame per facilitarne la conversione in R
    r_data_training = pd.DataFrame(r_data_training, columns=["log_ret"])
    r_data_testing = pd.DataFrame(r_data_testing, columns=["log_ret"])

    # Converti il DataFrame Pandas in un oggetto R
    r_data_training = pandas2ri.py2rpy(r_data_training)
    r_data_testing = pandas2ri.py2rpy(r_data_testing)


    # Passa il dato a R
    ro.globalenv['train_log_rets'] = r_data_training
    ro.globalenv['test_log_rets'] = r_data_testing
    

    # Scrivi lo script per calcolare il modello GARCH con la serie reale
    r_script = """
    
    # Fit GARCH(1,1) con i dati reali
    garch_model <- fGarch::garchFit(formula=~garch(1,1), data=train_log_rets, init.rec="mci", cond.dist="norm", algorithm="lbfgsb")
    summary(garch_model)
        
    #Evaluate autocorrelation of model residuals with Ljung-Box test
    #residuals <- residuals(model, standardize = TRUE)

    # Test di Ljung-Box sui residui
    #ljung_box_residuals <- Box.test(residuals, lag = 10, type = "Ljung-Box")
    
    #Evaluate accuracy of the model
    forecast_length <- nrow(test_log_rets)
    
    # Calcola la volatilità condizionata fittando il modello sul testing set
    forecasts <- fGarch::predict(garch_model, n.ahead=forecast_length)
    
    # Extract forecasted means and conditional variances
    forecasted_means <- forecasts$meanForecast
    #forecasted_variances <- forecasts$standardDeviation^2
    #forecasted_means
    
    #Convert test_log_rets to numeric vector
    test_log_rets <- test_log_rets$log_ret    

    
    # Evaluate the model with MAE, MSE, RMSE, MPE, MAPE, SMAPE, MASE, RMMSE
    MAE <- Metrics::mae(test_log_rets, forecasted_means) #Mean Absolute Error
    MAPE <- Metrics::mape(test_log_rets, forecasted_means) #Mean Absolute Percentage Error
    RMSE <- Metrics::rmse(test_log_rets, forecasted_means) #Root Mean Squared Error
    MSE <- RMSE^2 #Mean Squared Error
    SMAPE <- Metrics::smape(test_log_rets, forecasted_means) #Symmetric Mean Absolute Percentage Error
    MASE <-c(test_log_rets, forecasted_means) #Mean Absolute Scaled Error DOMANDA step size che ci metto?
    MPE <- mean((test_log_rets - forecasted_means) / test_log_rets)*100 #Mean Percentage Error
    
    rmsse_numerator <- sum((test_log_rets - forecasted_means)^2) / forecast_length
    rmsse_denominator <- sum(diff(test_log_rets)^2) / (forecast_length - 1)
    RMSSE <- sqrt(rmsse_numerator / rmsse_denominator) #Root Mean Squared Scaled Error
    
    
    
    #Extract coefficients of the model
    coefficients <- coef(garch_model)  
      
    """

    # Esecuzione del codice in R
    ro.r(r_script)
    
    #Print ljung box test results
    #print(ro.r('ljung_box_residuals'))
    
    #Returns mu, omega, alpha1, beta1
    coefficients = ro.r('coefficients')
    print("MU -> " + str(coefficients[0]))
    print("OMEGA -> " + str(coefficients[1]))
    print("ALPHA1 -> " + str(coefficients[2]))
    print("BETA1 -> " + str(coefficients[3]))
    print()
    
    #Returns MAE, MAPE, RMSE, MSE, MPE, SMAPE, MASE, RMMSE
    print("MAE -> " + str(ro.r('MAE')[0]))
    print("RMSE -> " + str(ro.r('RMSE')[0]))
    print("MSE -> " + str(ro.r('MSE')[0]))
    
    print("MPE -> " + str(ro.r('MPE')[0]))
    print("MAPE -> " + str(ro.r('MAPE')[0]))    
    print("SMAPE -> " + str(ro.r('SMAPE')[0]))
    
    print("MASE -> " + str(ro.r('MASE')[0]))
    print("RMSSE -> " + str(ro.r('RMSSE')[0]))
    

In [ ]:
compute_long_run_volatility('^SPX')

'''
OMEGA -> 9.596220100948595e-07
ALPHA1 -> 0.07713687415236392
BETA1 -> 0.9149867274327818
'''


'''
Standardised Residuals Tests:
                                 Statistic     p-Value
 Jarque-Bera Test   R    Chi^2   7.7015681 0.021263058 -> H0 è che i residui siano normali, la rigettiamo perche p<0.05
 Shapiro-Wilk Test  R    W       0.9935248 0.003018532 -> H0 è che i residui siano normali, la rigettiamo perche p<0.05
 Ljung-Box Test     R    Q(10)   6.2869703 0.790604543 -> H0 è che i residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R    Q(15)  10.9893952 0.753345768 -> H0 è che i residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R    Q(20)  23.0590353 0.285905087 -> H0 è che i residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R^2  Q(10)  14.2173008 0.163308600 -> H0 è che i quadrati dei residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R^2  Q(15)  19.2041441 0.204631699 -> H0 è che i quadrati dei residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R^2  Q(20)  21.6649474 0.358964724 -> H0 è che i quadrati residui siano scorrelati, la accettiamo perche p>0.05
 LM Arch Test       R    TR^2   15.2297226 0.229113370 -> H0 è che i i residui siano eteroschedastici, la accettiamo perche p>0.05
'''

'''
Accuracy:
MAE -> 0.006094533709048874
RMSE -> 0.008610649261663116
MSE -> 7.414328070737957e-05
MPE -> -87.3943129139289
MAPE -> 2.5240103117705095
SMAPE -> 1.5680391477240112
MASE -> -0.0187901349287447
RMSSE -> 0.8752686658213069
'''

In [ ]:
compute_long_run_volatility('^NDX')

'''
OMEGA -> 1.5333717379024792e-06
ALPHA1 -> 0.05178665160818798
BETA1 -> 0.9405498291075637
'''

'''
 Jarque-Bera Test   R    Chi^2   5.1105253 0.07767183 -> H0 è che i residui siano normali, la accettiamo perche p>0.05
 Shapiro-Wilk Test  R    W       0.9961994 0.07445377 -> H0 è che i residui siano normali, la accettiamo perche p>0.05
 Ljung-Box Test     R    Q(10)   7.3217410 0.69476141 -> H0 è che i residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R    Q(15)  12.8546429 0.61352425 -> H0 è che i residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R    Q(20)  21.2638981 0.38174870 -> H0 è che i residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R^2  Q(10)  15.1731763 0.12587456 -> H0 è che i quadrati dei residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R^2  Q(15)  20.7659377 0.14447586 -> H0 è che i quadrati dei residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R^2  Q(20)  24.2607337 0.23118830 -> H0 è che i quadrati residui siano scorrelati, la accettiamo perche p>0.05
 LM Arch Test       R    TR^2   18.1407399 0.11148352 -> H0 è che i i residui siano eteroschedastici, la accettiamo perche p>0.05
'''

'''
Accuracy:
MAE -> 0.008033379728990257
RMSE -> 0.011390500807927877
MSE -> 0.00012974350865540562
MPE -> 105.20046753081296
MAPE -> 1.0765264495006166
SMAPE -> 1.5913693108920905
MASE -> -0.0246935346579668
RMSSE -> 0.8172577792319691
'''

In [ ]:
compute_long_run_volatility('^RUT')

'''
OMEGA -> 4.048774895259176e-06
ALPHA1 -> 0.051128352743122475
BETA1 -> 0.9294291094425949
'''

'''
Standardised Residuals Tests:
                                Statistic   p-Value
 Jarque-Bera Test   R    Chi^2   2.980368 0.2253312 -> H0 è che i residui siano normali, la accettiamo perche p>0.05
 Shapiro-Wilk Test  R    W       0.996578 0.1176416 -> H0 è che i residui siano normali, la accettiamo perche p>0.05
 Ljung-Box Test     R    Q(10)   4.459829 0.9242289 -> H0 è che i residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R    Q(15)   9.566764 0.8460712 -> H0 è che i residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R    Q(20)  22.830264 0.2972056 -> H0 è che i residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R^2  Q(10)  11.275451 0.3364644 -> H0 è che i quadrati dei residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R^2  Q(15)  14.676889 0.4749321 -> H0 è che i quadrati dei residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R^2  Q(20)  20.998310 0.3972321 -> H0 è che i quadrati residui siano scorrelati, la accettiamo perche p>0.05
 LM Arch Test       R    TR^2   15.590459 0.2107218 -> H0 è che i i residui siano eteroschedastici, la accettiamo perche p>0.05
'''

'''
Accuracy:
MAE -> 0.01202009832952726
RMSE -> 0.016753480177713713
MSE -> 0.0002806790980650463
MPE -> 96.82370051310004
MAPE -> 0.9682370051310004
SMAPE -> 1.881695039220451
MASE -> -0.0164304752047055
RMSSE -> 0.8171991618433605
'''

In [ ]:
compute_long_run_volatility('NVDA')

'''
OMEGA -> 7.515112948238467e-05
ALPHA1 -> 0.03359987556656807
BETA1 -> 0.9050285576943499
'''

'''
Jarque-Bera Test   R    Chi^2  528.9313363 0.000000e+00 -> H0 è che i residui siano normali, la rigettiamo perche p<0.05 STRANO???
 Shapiro-Wilk Test  R    W        0.9679727 1.432350e-11 -> H0 è che i residui siano normali, la rigettiamo perche p<0.05
 Ljung-Box Test     R    Q(10)    8.1908486 6.102012e-01 -> H0 è che i residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R    Q(15)   14.0810954 5.193878e-01 -> H0 è che i residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R    Q(20)   19.5979460 4.833191e-01 -> H0 è che i residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R^2  Q(10)    2.0114543 9.962516e-01 -> H0 è che i quadrati dei residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R^2  Q(15)    3.6594247 9.986565e-01 -> H0 è che i quadrati dei residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R^2  Q(20)    4.3375827 9.999101e-01 -> H0 è che i quadrati residui siano scorrelati, la accettiamo perche p>0.05
 LM Arch Test       R    TR^2     2.0699933 9.992908e-01 -> H0 è che i i residui siano eteroschedastici, la accettiamo perche p>0.05
'''

'''
Accuracy:
MAE -> 0.02115490886540065
RMSE -> 0.02576324458913306
MSE -> 0.0006637447717594938
MPE -> 93.87573217972553
MAPE -> 0.9387573217972554
SMAPE -> 1.5955679847653488
MASE -> -0.0483739837112597
RMSSE -> 0.7479291550877064
'''

In [ ]:
compute_long_run_volatility('JNJ')

'''
OMEGA -> 1.2719627293334906e-05
ALPHA1 -> 0.006837376415740562
BETA1 -> 0.874687992060343
'''

'''
Standardised Residuals Tests:
                                  Statistic      p-Value
 Jarque-Bera Test   R    Chi^2  332.3234645 0.000000e+00 -> H0 è che i residui siano normali, la rigettiamo perche p<0.05
 Shapiro-Wilk Test  R    W        0.9665474 6.909536e-12 -> H0 è che i residui siano normali, la rigettiamo perche p<0.05
 Ljung-Box Test     R    Q(10)    9.5335088 4.823270e-01 -> H0 è che i residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R    Q(15)   11.9041086 6.862723e-01 -> H0 è che i residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R    Q(20)   14.1130944 8.247158e-01 -> H0 è che i residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R^2  Q(10)   13.4230151 2.009725e-01 -> H0 è che i quadrati dei residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R^2  Q(15)   17.0933117 3.133169e-01 -> H0 è che i quadrati dei residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R^2  Q(20)   19.2805051 5.036658e-01 -> H0 è che i quadrati residui siano scorrelati, la accettiamo perche p>0.05
 LM Arch Test       R    TR^2    15.9016300 1.957832e-01 -> H0 è che i i residui siano eteroschedastici, la accettiamo perche p>0.05
'''

'''
Accuracy:
MAE -> 0.006185113818306013
RMSE -> 0.007859111563955272
MSE -> 6.176563457469547e-05
MPE -> 102.94998142322815
MAPE -> 1.0294998142322815
SMAPE -> 1.9166124924434742
MASE -> -0.0046806575162836
RMSSE -> 0.6567625313870091
'''

In [ ]:
compute_long_run_volatility('XOM')

'''
OMEGA -> 8.603457755518873e-07
ALPHA1 -> 0.024873336478831613
BETA1 -> 0.9715906060158603
'''

'''
Standardised Residuals Tests:
                                 Statistic      p-Value
 Jarque-Bera Test   R    Chi^2  20.8895352 2.910014e-05 -> H0 è che i residui siano normali, la rigettiamo perche p<0.05
 Shapiro-Wilk Test  R    W       0.9936667 3.555295e-03 -> H0 è che i residui siano normali, la rigettiamo perche p<0.05
 Ljung-Box Test     R    Q(10)  11.7915249 2.992513e-01 -> H0 è che i residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R    Q(15)  20.8009089 1.433119e-01 -> H0 è che i residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R    Q(20)  27.9673765 1.101739e-01 -> H0 è che i residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R^2  Q(10)   7.7976889 6.485908e-01 -> H0 è che i quadrati dei residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R^2  Q(15)  15.2227726 4.354927e-01 -> H0 è che i quadrati dei residui siano scorrelati, la accettiamo perche p>0.05
 Ljung-Box Test     R^2  Q(20)  19.4795547 4.908793e-01 -> H0 è che i quadrati residui siano scorrelati, la accettiamo perche p>0.05
 LM Arch Test       R    TR^2   11.3263406 5.011776e-01 -> H0 è che i i residui siano eteroschedastici, la accettiamo perche p>0.05
'''

'''
Accuracy:
MAE -> 0.008742229965304483
RMSE -> 0.011766014883476403
MSE -> 0.00013843910623818824
MPE -> 103.7901895069331
MAPE -> 1.0559375770067392
SMAPE -> 1.5949755700315245
MASE -> 0.0007709548367103
RMSSE -> 0.6587460673532036
'''